In [ ]:
# for high bandwidth model downloading
!pip install hf_transfer
import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction

from datasets import load_dataset, Split

from typing import Dict

# target_model_id = 'NousResearch/Llama-2-7b-hf'
# supervisor_model_id = 'NousResearch/Llama-2-7b-hf'

target_model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
supervisor_model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [ ]:
target_tokenizer = AutoTokenizer.from_pretrained(target_model_id)
target_model = AutoModelForCausalLM.from_pretrained(target_model_id)

supervisor_tokenizer = AutoTokenizer.from_pretrained(supervisor_model_id)
supervisor_model = AutoModelForCausalLM.from_pretrained(supervisor_model_id)

In [ ]:
# load TruthfulQA dataset. VALIDATION is the only available split
all_data = load_dataset(path='truthful_qa', name='generation', split=Split.VALIDATION).train_test_split(train_size=0.66, shuffle=True, seed=42)

In [ ]:
def target_tokenize(examples):
    return target_tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
def compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    predictions, labels = eval_pred
    # TODO: measure alignment using `supervisor_model`
    return {}

In [ ]:
training_args = TrainingArguments(output_dir='test_trainer', evaluation_strategy='epoch')

trainer = Trainer(
    model=target_model,
    args=TrainingArguments,
    train_dataset=all_data[Split.TRAIN],
    eval_dataset=all_data[Split.TEST],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()